VQEAC
=============================

In [1]:
import functools 

import pennylane as qml
from pennylane import numpy as np
import optax
import jax

h2_dataset = qml.data.load("qchem", molname="H2", bondlength=0.742, basis="STO-3G")
h2 = h2_dataset[0]
H, qubits = h2.hamiltonian, len(h2.hamiltonian.wires)

In [2]:
h2.hf_state

tensor([1, 1, 0, 0], dtype=int64, requires_grad=True)

## VQE
Each VQE needs the following:
- An Ansatz
- Loss function

In [3]:
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is    (-0.22250914236600539) [Z2]
+ (-0.22250914236600539) [Z3]
+ (-0.09963387941370971) [I0]
+ (0.17110545123720225) [Z1]
+ (0.17110545123720233) [Z0]
+ (0.12051027989546245) [Z0 Z2]
+ (0.12051027989546245) [Z1 Z3]
+ (0.16584090244119712) [Z0 Z3]
+ (0.16584090244119712) [Z1 Z2]
+ (0.16859349595532533) [Z0 Z1]
+ (0.1743207725924201) [Z2 Z3]
+ (-0.04533062254573469) [Y0 Y1 X2 X3]
+ (-0.04533062254573469) [X0 X1 Y2 Y3]
+ (0.04533062254573469) [Y0 X1 X2 Y3]
+ (0.04533062254573469) [X0 Y1 Y2 X3]


## Begin training
Let's set some expectation for the optimization process. Thankfully, $H_2$ is well studied and we have all we need in the `dataset` library to know the ground truth

### Ansatz

Before any run, we can assume that the Jordan Wigner representation `[1 1 0 0]` has the lowest energy. Let's calculate that

In [4]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def circuit_exptected():
    qml.BasisState(h2.hf_state, wires = range(qubits))
    for op in h2.vqe_gates:
        qml.apply(op)
    return qml.state()

Since the Hamiltonian doesn't have a canonical representation, we have to create a work around

The idea herer is, for `Z(1)`, we convert into `I(0) @ Z(1) @ I(1) @ I(2)`, and do the same for every Hamiltonian operator

In [5]:
def hamiltonian_to_matrix(hamiltonian, wires: qml.wires.Wires):
    matrix = 0+0j
    set_wires = set(wires)
    list_wires = list(wires)
    for coeff, op in zip(hamiltonian.coeffs, hamiltonian.ops):
        missing_wires = set_wires - set(op.wires)
        padded_ops = op.obs if isinstance(op, qml.operation.Tensor) else [op]        
        for wire in list_wires:
            if wire in missing_wires:
                padded_ops.append(qml.Identity(wire))
        padded_ops.sort(key=lambda x: x.wires.tolist())
        padded_ops = [op.matrix() for op in padded_ops]        
        matrix += coeff * functools.reduce(lambda a, b: np.kron(a, b), padded_ops)
    return matrix

In [6]:
print(f"HF state: {h2.hf_state}")
circuit_exptected()

HF state: [1 1 0 0]


array([ 0.        +0.j,  0.        +0.j,  0.        +0.j, -0.13619566+0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.99068196+0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])

In [7]:
circuit_exptected()

array([ 0.        +0.j,  0.        +0.j,  0.        +0.j, -0.13619566+0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.99068196+0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])

In [8]:
circuit_exptected().conj().T @ hamiltonian_to_matrix(h2.hamiltonian, h2.hamiltonian.wires) @ circuit_exptected()

tensor(-1.13637658+0.j, requires_grad=True)

Taking the superposition with themselves and the higher/lower energy level (excite/de-excite). Note that in `h2.vqe_gates` we already have the value for $\theta$

In [9]:
print(qml.draw(circuit_exptected)())

0: ─╭|Ψ⟩─╭G²(0.27)─┤  State
1: ─├|Ψ⟩─├G²(0.27)─┤  State
2: ─├|Ψ⟩─├G²(0.27)─┤  State
3: ─╰|Ψ⟩─╰G²(0.27)─┤  State


We would define the same circuit but without the $\theta$. Given 2 $H$ and 4 qubits, after a double excitation, the HF is the superposition of the states
$$\alpha\ket{1100}+\beta\ket{0011}:=\cos(\theta)\ket{1100}-\sin(\theta)\ket{0011}$$

[comment]: # ($\alpha\ket{110000}+\beta\ket{001100}+\gamma\ket{000011}$ this is H3)


In [10]:
@qml.qnode(dev)
def circuit(param):
    qml.BasisState(h2.hf_state, wires=range(qubits))
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
    return qml.state()

### Define the lost function
Remember that the lost function is the second ingredient. We use the first two equations in [this paper](https://www.nature.com/articles/s41524-023-00965-1)
$$\left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {\hat H} \right|{\Psi}\left( {{{\mathbf{\theta }}}} \right)} \right\rangle$$

$$C_1\left( {{{\mathbf{\theta }}}} \right) = \left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {\hat H} \right|{\Psi}\left( {{{\mathbf{\theta }}}} \right)} \right\rangle + \beta \left| {\left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {{\Psi}_0} \right.} \right\rangle } \right|^2$$

We can then define a lost function



At first sight, it might raises some eyebrow for someone who is from a ML background, because we define the loss function based on the predicted and the groundtruth. However we do not have any groundtruth value here. In this context, a loss function is just a function that we want to minimize.


Now we proceed to optimize the variational parameters

In [11]:
def loss_fn_1(theta):
    """
    Pure expectation value
    """
    state = circuit(theta)
    return state.conj().T @ hamiltonian_to_matrix(h2.hamiltonian, h2.hamiltonian.wires) @ state

def loss_fn_2(theta, beta):
    """
    Expectation value
    """
    state = circuit(theta)
    return state.conj().T @ hamiltonian_to_matrix(h2.hamiltonian, h2.hamiltonian.wires) @ state + beta*(state.conj().T @ h2.hf_state)**2

In [12]:
theta = np.array(0.)

# store the values of the cost function
energy = [loss_fn_1(theta)]
conv_tol = 1e-6
max_iterations = 100
opt = optax.sgd(learning_rate=0.4)

# store the values of the circuit parameter
angle = [theta]

opt_state = opt.init(theta)

for n in range(max_iterations):
    gradient = jax.grad(loss_fn_1)(theta)
    updates, opt_state = opt.update(gradient, opt_state)
    theta = optax.apply_updates(theta, updates)
    
    angle.append(theta)
    energy.append(loss_fn_1(theta))

    conv = np.abs(energy[-1] - energy[-2])

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

C:\code\vqeac\.venv\lib\site-packages\pennylane\math\utils.py:227: UserWarning: Contains tensors of types {'jax', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


ValueError: Computing the gradient of circuits that return the state with the parameter-shift rule gradient transform is not supported, as it is a hardware-compatible method.

In [ ]:
theta

In [ ]:
circuit(theta)

It is bigger than the minimum expected energy above

## Running the VQD optimization

According to equation 2 from [this paper](https://www.nature.com/articles/s41524-023-00965-1),